In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import pylab
import glob, os
import scipy.stats as stats
from scipy.stats import gaussian_kde
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import timeit
%matplotlib inline

In [2]:
# Format to remove scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
DATA_PATH = 'R:/Angela/fast_trips/KCMetro'
FLEET = os.path.join(DATA_PATH, 'fleet_NEW.csv')
ROUTE = os.path.join(DATA_PATH, '151AllTrips.csv')
DATE_LIST = ['2015_04_28.csv', '2015_04_29.csv', '2015_04_30.csv', '2015_05_05.csv', '2015_05_06.csv', '2015_05_07.csv']

# Read-in Data

In [ ]:
APC = {}
i = 1
for csv_file in DATE_LIST:
    APC_df = pd.read_csv(os.path.join(DATA_PATH, csv_file))
    APC_df['date_id'] = i
    APC[i] = APC_df
    i += 1

In [ ]:
for data in APC.values():
    print len(data)

# Get Fleet Type & Capacity Info Ready

In [6]:
fleet = pd.read_csv(FLEET)

In [7]:
fleet.head()

,fleet_a,fleet_b,Seated Capcity,Standing Capacity,Total Capacity,Low_Floor
0,1100,1194,20,28,48,0
1,2300,2399,53,62,115,0
2,2400,2573,53,62,115,1
3,2600,2812,53,62,115,1
4,2870,2899,53,62,115,1


In [8]:
def fleet_type(data):
    #get fleet type info
    fleet_dict = {}
    floor_dict = {}
    for ele in np.unique(data.VEHICLE_ID):
        for a in fleet.fleet_a:
            b = int(fleet[fleet.fleet_a == a].fleet_b)
            if a <= ele <= b:
                fleet_dict[ele] = int(fleet[fleet.fleet_a == a]['Total Capacity'])
                floor_dict[ele] = int(fleet[fleet.fleet_a == a]['Low_Floor'])
                #print fleet[fleet.fleet_a == a]
    print 'fleet_dict length is:', len(fleet_dict), len(floor_dict)
    print 'unique vehicle number is:', len(np.unique(data.VEHICLE_ID))
    #link fleet info to vehicles
    for ele in np.unique(data.VEHICLE_ID):
        if ele in fleet_dict:
            continue
        else:
            print ele
    
    data['capacity'] = data['VEHICLE_ID'].map(fleet_dict)
    data['low_floor'] = data['VEHICLE_ID'].map(floor_dict)
    print 'current data shape is:', data.shape

    return data

In [9]:
fleet_data = fleet_type(data)

fleet_dict length is: 1210 1210
unique vehicle number is: 1211
9536
current data shape is: (585595, 34)


In [10]:
fleet_data.columns

Index([u'DAYCODE', u'TRIP_ID', u'PATTERN_ID', u'BLK', u'RTE', u'DIR',
       u'SCH_ST_MIN', u'OPD_DATE', u'VEHICLE_ID', u'BUS_TYPE', u'ENGINE',
       u'NUM_DOORS', u'STOP_SEQ', u'STOP_NAME', u'STOP_ID', u'GPS_LONGITUDE',
       u'GPS_LATITUDE', u'DOORS_OPEN', u'DOOR_OPEN_SEC', u'SCH_STOP_SEC',
       u'ACT_STOP_ARR', u'ACT_STOP_DEP', u'DWELL_SEC', u'ACT_HOUR',
       u'ACT_STOP_ARR_TM', u'ACT_STOP_DEP_TM', u'APC_VEH', u'ONS', u'OFFS',
       u'LOAD', u'STOP_TYPE', u'date_id', u'capacity', u'low_floor'],
      dtype='object')

In [11]:
fleet_data.head()

,DAYCODE,TRIP_ID,PATTERN_ID,BLK,RTE,DIR,SCH_ST_MIN,OPD_DATE,VEHICLE_ID,BUS_TYPE,...,ACT_STOP_ARR_TM,ACT_STOP_DEP_TM,APC_VEH,ONS,OFFS,LOAD,STOP_TYPE,date_id,capacity,low_floor
0,0,27133771,21007011,701,7,S,252,7-May-15,4238,Artic,...,4:10:56,4:10:56,N,nan,nan,nan,T,6,92.000,0.000
1,0,27133771,21007011,701,7,S,252,7-May-15,4238,Artic,...,4:11:49,4:11:56,N,nan,nan,nan,S,6,92.000,0.000
2,0,27133771,21007011,701,7,S,252,7-May-15,4238,Artic,...,4:12:19,4:12:19,N,nan,nan,nan,ST,6,92.000,0.000
3,0,27133771,21007011,701,7,S,252,7-May-15,4238,Artic,...,4:12:35,4:12:47,N,nan,nan,nan,S,6,92.000,0.000
4,0,27133771,21007011,701,7,S,252,7-May-15,4238,Artic,...,4:13:01,4:13:01,N,nan,nan,nan,S,6,92.000,0.000


In [12]:
test = fleet_data[['low_floor', 'VEHICLE_ID']]

In [13]:
np.unique(test['VEHICLE_ID'])

array([1100, 1101, 1103, ..., 9811, 9812, 9813], dtype=int64)

# Get Route Type Info Ready

In [14]:
route = pd.read_csv(ROUTE)

In [15]:
rapid_list = [671, 672, 673, 674, 675, 676]
for rte in rapid_list:
    route.loc[route['Rte'] == rte, 'E/L'] = 'R'
dict_route_type = route.set_index('TRIP_ID').to_dict()['E/L']

In [16]:
# Pre-check: there are TRIP_IDs not included in the 151ALLTrips.csv

def check_lefttrips(data):
    j = 0
    left_rte = {}
    for trip in np.unique(data.TRIP_ID):
        if trip in dict_route_type:
            continue
        else:
            #print trip
            left_rte[trip] = int(np.unique(data[data.TRIP_ID == trip]['RTE']))
            #print len(data[data.TRIP_ID == trip])
            j = len(data[data.TRIP_ID == trip]) + j
    print left_rte
    print 'APC records lenth', j

check_lefttrips(fleet_data)  
    


{28345597: 550}
APC records lenth 37


In [17]:
def route_type(data):
    data['Route Type'] = data['TRIP_ID'].map(dict_route_type) 
    print data['Route Type'] .isnull().sum()
    print data.shape
    data = data.dropna(subset=['Route Type'])
    #data['Route Type'] = data['Route Type'].dropna()
    print data.shape
    
    return data

In [18]:
#route_data = route_type(fleet_data)

In [19]:
# Post-check

'''
print np.unique(route_data['Route Type'])
print route_data.columns
'''

"\nprint np.unique(route_data['Route Type'])\nprint route_data.columns\n"

# Get all I need 

In [20]:
def content_need(data):
    data = data[['TRIP_ID', 'RTE', 'VEHICLE_ID', 'BUS_TYPE', 'NUM_DOORS', 
                 'STOP_SEQ', 'STOP_ID','DWELL_SEC', 'DOOR_OPEN_SEC', 
                 'ONS', 'OFFS', 'LOAD', 'date_id', 'capacity', 'Route Type', 'low_floor']]
    return data

In [21]:
#my_data = content_need(route_data)

# Data Clean Equations Preparation

# Step1: Clean Outlines Data

In [22]:
def clean_data(data):
    start = timeit.default_timer()
    
    #get fleet and vehicle type info
    data = fleet_type(data)
    data = route_type(data)
    data = content_need(data)
    
    # Before cleaning records, get the load data from the previous stop
    data = data.sort(['TRIP_ID', 'STOP_SEQ'])
    data['pre_load'] = data['LOAD'].shift()
    
    # Remove the corner data (fist and last stops) before the data was disordered by other cleaning processes
    def del_stop(data):
        return data.iloc[1:-1]
    data = data.groupby('TRIP_ID', group_keys=False).apply(del_stop)
    print 'clean first and last stops', data.shape
    
    #data = data[pd.notnull(data['pre_load'])]
    #print 'clean invalid pre_load', data.shape
    
    # 0< Dwell time < 120 
    data = data[data['DOOR_OPEN_SEC'] != 0]
    print 'cleaned zero dwell', data.shape
    data = data[data['DOOR_OPEN_SEC'] > 0]
    data = data[data['DOOR_OPEN_SEC'] <= 90]
    #data = data[data['DOOR_OPEN_SEC'] <= 120]
    print 'cleaned large dwell', data.shape
    
    # On + off != 0
    data = data[pd.notnull(data['ONS'])]
    data = data[data['ONS'] + data['OFFS'] != 0]
    print 'cleaned the invalid or zero boarding+alighting records:', data.shape
    
    # valid bus type
    data = data[data.BUS_TYPE != '??']
    print 'cleaned unknow bus type', data.shape
    
    # delete all downtown tunnel stop records
    dict_stop_id = dict((el,1) for el in [455, 456, 501, 502, 532, 533, 565, 566, 621, 622, 623, 624, 
                                      1082, 1083, 1084, 1086, 1108, 1109, 1121, 1122, 1192, 1193])
    data['downtown_stop'] = data['STOP_ID'].map(dict_stop_id)
    data.downtown_stop = data.downtown_stop.fillna(0)
    data = data[data.downtown_stop != 1]
    print 'cleaned downtown tunnel', data.shape
    stop = timeit.default_timer()
    print 'removed invalid data:', stop - start
    print '---------------------------------------------'
    
    return data

# Step2: Prepare Extra Basic Variables

In [1]:
def get_x_y(data): 
    start = timeit.default_timer()
    
    # Bus type: Articulated or Standard bus; TWo or Three doors; fleet
    data['Artic'] = data['BUS_TYPE'].map({'Artic': 1, 'Standard': 0})
    data['Std'] = data['BUS_TYPE'].map({'Artic': 0, 'Standard': 1})
    
    # low floor
    data['high_floor'] = data['low_floor'].map({1:0, 0:1})
    
    # number of doors
    # fix the data error: there are some one-door vehicles
    for vno in np.unique(data['VEHICLE_ID']):
        if vno > 1100 and vno < 1199:
            data.loc[data['VEHICLE_ID']==vno, 'NUM_DOORS'] = 1
            #w.loc[w.female == 'female', 'female'] = 1
    data['twodoors'] = data['NUM_DOORS'].map({1:0, 2: 1, 3: 0})
    data['threedoors'] = data['NUM_DOORS'].map({1:0, 2: 0, 3: 1})
    
    
    # Route Type, create dummie variables for each route type
    just_dummies_route = pd.get_dummies(data['Route Type'])
    step_1 = pd.concat([data, just_dummies_route], axis=1)
    # Avoid dummie variable trap
    step_1.drop(['L'], inplace=True, axis=1)
    data = step_1
    stop = timeit.default_timer()
    print 'add veh&route info:', stop - start
    
    start = timeit.default_timer()
    # Passenger activity
    data['ON+OFF'] = data['ONS'] + data['OFFS']
    data['ma_pasg'] = data[['ONS', 'OFFS']].max(axis=1)
    data['ab_pasg'] = (data['ONS'] - data['OFFS']).abs()
    data['pre_standees']= data['pre_load'] - 0.60 * data['capacity']
    data['pre_crowding']= data.apply(lambda x: x['pre_standees'] > 0, axis=1).map({False: 0, True: 1})
    data['friction'] = ((data['ONS'] + data['OFFS'] + (data['pre_standees']).abs()) * data['pre_crowding']).abs()
    print data.shape
    
    
    # Interactive vars: second set
    data['on_art'] = data['ONS']*data['Artic']
    data['off_art'] = data['OFFS']*data['Artic']
    data['on_threedoors'] = data['ONS']*data['threedoors']
    data['off_threedoors'] = data['OFFS']*data['threedoors']
    data['on_twodoors'] = data['ONS']*data['twodoors']
    data['off_twodoors'] = data['OFFS']*data['twodoors']
    data['on_low'] = data['ONS']*data['low_floor']
    data['off_low'] = data['OFFS']*data['low_floor']
    data['on_rapid'] = data['ONS']*data['R']
    data['off_rapid'] = data['OFFS']*data['R']
    data['on_express'] = data['ONS']*data['E']
    data['off_express'] = data['OFFS']*data['E']
    
    
    print data.shape
    stop = timeit.default_timer()
    print 'add correlation variables:', stop - start
    
    return data

# Step3: Prepare Extra Control Variable: Vehnicle ID

In [81]:
# Create dummie variables for bus id 
def create_vehID(data):
    start = timeit.default_timer()
    print 'data shape:', data.shape
    data['vehno_date'] = data.VEHICLE_ID.astype(str) + '_' + data.date_id.astype(str)
    print len(np.unique(data.vehno_date))
    just_dummies_veh = pd.get_dummies(data['vehno_date'])
    data = pd.concat([data, just_dummies_veh], axis=1)
    stop = timeit.default_timer()
    print 'data shape:', data.shape
    print 'add vehid variables:', stop - start
    
    return data

# Ok, Let's Clean It! 

In [82]:
df_base = pd.DataFrame(columns = ['TRIP_ID', 'RTE', 'VEHICLE_ID', 'BUS_TYPE', 'NUM_DOORS',
                                  'STOP_SEQ', 'STOP_ID', 'DWELL_SEC', 'DOOR_OPEN_SEC', 
                                  'ONS', 'OFFS', 'LOAD', 'date_id', 'capacity', 
                                  'Route Type', 'pre_load','downtown_stop', 'low_floor'])
print len(df_base)
for my_data in APC.values():
    print len(my_data)
    
    my_data = clean_data(my_data)
    print my_data.columns
    df_base = pd.concat([df_base, my_data])
    print len(df_base)


0
581968
fleet_dict length is: 1223 1223
unique vehicle number is: 1223
current data shape is: (581968, 35)
39
(581968, 35)
(581929, 35)
clean first and last stops (560931, 17)
cleaned zero dwell (209743, 17)
cleaned large dwell (199886, 17)
cleaned the invalid or zero boarding+alighting records: (60693, 17)
cleaned unknow bus type (59413, 17)
cleaned downtown tunnel (57335, 18)
removed invalid data: 35.3354285449
---------------------------------------------
Index([u'TRIP_ID', u'RTE', u'VEHICLE_ID', u'BUS_TYPE', u'NUM_DOORS',
       u'STOP_SEQ', u'STOP_ID', u'DWELL_SEC', u'DOOR_OPEN_SEC', u'ONS',
       u'OFFS', u'LOAD', u'date_id', u'capacity', u'Route Type', u'low_floor',
       u'pre_load', u'downtown_stop'],
      dtype='object')
57335
582850
fleet_dict length is: 1219 1219
unique vehicle number is: 1219
current data shape is: (582850, 35)
403
(582850, 35)
(582447, 35)
clean first and last stops (561541, 17)
cleaned zero dwell (211099, 17)
cleaned large dwell (201124, 17)
cleaned 

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [83]:
df_base.columns

Index([u'BUS_TYPE', u'DOOR_OPEN_SEC', u'DWELL_SEC', u'LOAD', u'NUM_DOORS',
       u'OFFS', u'ONS', u'RTE', u'Route Type', u'STOP_ID', u'STOP_SEQ',
       u'TRIP_ID', u'VEHICLE_ID', u'capacity', u'date_id', u'downtown_stop',
       u'low_floor', u'pre_load'],
      dtype='object')

In [84]:
step2 = get_x_y(df_base)

add veh&route info: 0.449573954538
(333819, 31)
(333819, 43)
add correlation variables: 4.41261389185


In [85]:
step2 = step2[step2['DOOR_OPEN_SEC'] >0]

In [86]:
np.unique(step2['DOOR_OPEN_SEC'])

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.])

In [87]:
step2.columns

Index([u'BUS_TYPE', u'DOOR_OPEN_SEC', u'DWELL_SEC', u'LOAD', u'NUM_DOORS',
       u'OFFS', u'ONS', u'RTE', u'Route Type', u'STOP_ID', u'STOP_SEQ',
       u'TRIP_ID', u'VEHICLE_ID', u'capacity', u'date_id', u'downtown_stop',
       u'low_floor', u'pre_load', u'Artic', u'Std', u'high_floor', u'twodoors',
       u'threedoors', u'E', u'R', u'ON+OFF', u'ma_pasg', u'ab_pasg',
       u'pre_standees', u'pre_crowding', u'friction', u'on_art', u'off_art',
       u'on_threedoors', u'off_threedoors', u'on_twodoors', u'off_twodoors',
       u'on_low', u'off_low', u'on_rapid', u'off_rapid', u'on_express',
       u'off_express'],
      dtype='object')

In [98]:
step3 = step2[step2['ON+OFF'] < 30]

In [100]:
print len(step2)
print len(step3)

333817
332386


In [91]:
#test.head()

In [101]:
my_project = step3

# Create Training (70%) and Validating (30%) datasets 

In [102]:
import os
os.getcwd()

'C:\\Users\\Angela\\Documents\\GitHub\\fast-trips_dwell_time_model'

In [103]:
msk = np.random.rand(len(my_project)) < 0.7
df_train = my_project[msk]
df_test = my_project[~msk]

In [104]:
df_train.columns

Index([u'BUS_TYPE', u'DOOR_OPEN_SEC', u'DWELL_SEC', u'LOAD', u'NUM_DOORS',
       u'OFFS', u'ONS', u'RTE', u'Route Type', u'STOP_ID', u'STOP_SEQ',
       u'TRIP_ID', u'VEHICLE_ID', u'capacity', u'date_id', u'downtown_stop',
       u'low_floor', u'pre_load', u'Artic', u'Std', u'high_floor', u'twodoors',
       u'threedoors', u'E', u'R', u'ON+OFF', u'ma_pasg', u'ab_pasg',
       u'pre_standees', u'pre_crowding', u'friction', u'on_art', u'off_art',
       u'on_threedoors', u'off_threedoors', u'on_twodoors', u'off_twodoors',
       u'on_low', u'off_low', u'on_rapid', u'off_rapid', u'on_express',
       u'off_express'],
      dtype='object')

In [105]:
df_train.to_csv(r'C:\Users\Angela\Documents\Github\fast-trips_dwell_time_model\KC_Metro_Inner_dataset\KC_df_train.csv')
df_test.to_csv(r'C:\Users\Angela\Documents\Github\fast-trips_dwell_time_model\KC_Metro_Inner_dataset\KC_df_test.csv')

In [106]:
print len(df_train)
print len(df_test)

232843
99543


In [ ]:
print 'Congrats! go for a quick run in Puget Sound!'